# Jupyter Notebook

## Introduction

+ What is Jupyter notebook?

+ Why do we want to use Jupyter notebook?

+ Features?

## Setting up Jupyter notebook

+ Load module ``anaconda/py36/5.0.1`` and activate the ``prace`` environment.
```
module load anaconda/py36/5.0.1
source activate prace
```

+ Create your password for Jupyter notebook **(Security measure #1)** 
```
jupyter notebook password
```

+ Generate a self-signed certificate with ``openssl`` **(Security measure #2)** 
```
openssl req -x509 -nodes -days 30 -newkey rsa:2048 -keyout mykey.key -out mycert.pem
```

+ Start Jupyter notebook with port, host IP, and secure protocol
```
ipnport=$(shuf -i8000-9999 -n1)
ipnip=$(hostname -i)
jupyter-notebook --no-browser --port=$ipnport --ip=$ipnip --certfile=mycert.pem --keyfile mykey.key
```

+ Jupyter notebook will print output like
```
The Jupyter Notebook is running at:
https://<IP.address>:<port>/
```

+ Copy the **https** link to your browser. Your browser will likely complain that your self-signed certificate is not secure or not recognized. Tell it to proceed anyways.


+ The notebook is ready for use after entering your password!